In [106]:
import pandas as pd
import numpy as np
import _pickle as cPickle
from datetime import timedelta
from tqdm import tqdm, tqdm_notebook
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from itertools import chain
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
%matplotlib inline

In [107]:
df1 = pd.read_csv('../custom_data/Feature_Engineering/for_cloud.csv')
df1.dropna(axis = 0, how = 'any', inplace = True)
df1.isna().sum()

df1['shifted_close'] = df1.groupby('ticker')['Close'].shift(-1)
df1['tgt'] = (df1.shifted_close - df1.Close).div(df1.shifted_close) * 100
df1['target'] = 0
df1.loc[(df1['tgt'] >= 0), 'target'] = 1
df1.loc[(df1['tgt'] < 0), 'target'] = 0
df1.drop(['shifted_close', 'tgt'], axis = 1 , inplace = True)

In [108]:
df1['Date_dt'] = pd.to_datetime(df1['Date'], format='%Y%m%d', errors='ignore')
df1['year'] = pd.DatetimeIndex(df1['Date']).year
df1['month'] = pd.DatetimeIndex(df1['Date']).month
df1.drop(['Date', 'Name'], axis = 1, inplace = True)

In [109]:
df1.head()

,ticker,Adj Close,Close,High,Low,Open,Volume,Sector,target,Date_dt,year,month
0,AAL,7.405228,7.73,7.87,7.48,7.48,4194100.0,Industrials,1,2008-12-31,2008,12
1,AAL,8.037499,8.39,8.48,7.67,7.73,5167000.0,Industrials,0,2009-01-02,2009,1
2,AAL,7.980019,8.33,8.39,7.96,8.38,3457100.0,Industrials,1,2009-01-05,2009,1
3,AAL,8.679349,9.06,9.21,8.13,8.15,5731000.0,Industrials,1,2009-01-06,2009,1
4,AAL,8.698509,9.08,9.47,8.66,8.66,5468900.0,Industrials,1,2009-01-07,2009,1


In [110]:
embedding_lengths = [100, 2, 2, 3]
encodings = {}
categorical_cols = ['ticker', 'Sector', 'year', 'month']

In [111]:
def get_xy(df):
    y = df['target']
    x = get_x(df)
    
    return x, y

def label_encode(series, min_counts=2):
    vc = series.value_counts()
    #reserve 0 for unknown
    le = {c : i+1 for i, c in enumerate(vc.index[vc > min_counts])}
    le['UNKN'] = 0
    return le

def get_encodings(df, cat_cols):
    if len(encodings) == 0:
        for col in cat_cols:
            encodings[col] = label_encode(df[col])
    return encodings

def map_encodings(df, cat_cols, encs):
    for col in cat_cols:
        df[col] = df[col].map(encs[col]).fillna(0).astype(int)
        
def get_x(df, isTrain=True):
    encodings = get_encodings(df, categorical_cols)
    map_encodings(df, categorical_cols, encodings) 
    if isTrain:
        cols_to_drop = ['target']#, 'Date_dt']
    else: 
        cols_to_drop = []#['Date_dt']
    
    df.drop(columns=cols_to_drop, inplace=True)
        
    return df

In [112]:
X, y = get_xy(df1)

In [113]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1168477 entries, 0 to 1254419
Data columns (total 11 columns):
ticker       1168477 non-null int32
Adj Close    1168477 non-null float64
Close        1168477 non-null float64
High         1168477 non-null float64
Low          1168477 non-null float64
Open         1168477 non-null float64
Volume       1168477 non-null float64
Sector       1168477 non-null int32
Date_dt      1168477 non-null object
year         1168477 non-null int32
month        1168477 non-null int32
dtypes: float64(6), int32(4), object(1)
memory usage: 89.1+ MB


In [114]:
#get all the numeric columns
num_cols = [x for x in X.columns if x not in categorical_cols]

#remove assetCode from num_cols
num_cols = [x for x in num_cols if x not in ['ticker', 'Date_dt']]

In [115]:
def scale_numeric(df):
    df[num_cols] = df[num_cols].fillna(0)

    scaler = StandardScaler()
    
    #need to do this due to memory contraints
    for i in range(0, len(num_cols), 4):
        cols = num_cols[i:i + 3]
        df[cols] = scaler.fit_transform(df[cols].astype(float))
        
def scale_numeric_min_max(df):
    df[num_cols] = df[num_cols].fillna(0)

    groups = df.groupby('ticker')
    mini = groups[num_cols].transform('min')
    maxi = groups[num_cols].transform('max')
    #std = groups[num_cols].transform(np.std)
    
    for col in num_cols:
        df[col] = (df[col] - mini[col]).div(maxi[col] - mini[col])
scale_numeric_min_max(X)
        
#scale_numeric(X)

In [116]:
X.head()

,ticker,Adj Close,Close,High,Low,Open,Volume,Sector,Date_dt,year,month
0,311,0.098495,0.101947,0.101421,0.100143,0.096496,0.022222,4,2008-12-31,11,10
1,311,0.109781,0.113628,0.112125,0.103541,0.100898,0.029344,4,2009-01-02,9,9
2,311,0.108755,0.112566,0.110546,0.108727,0.112344,0.016827,4,2009-01-05,9,9
3,311,0.121238,0.125487,0.124934,0.111767,0.108294,0.033472,4,2009-01-06,9,9
4,311,0.121580,0.125841,0.129496,0.121245,0.117274,0.031554,4,2009-01-07,9,9


In [117]:
#X[X['Date_dt']> '2018-01-01'].index

X_train = X[X['Date_dt']<= '2017-01-01']
X_test = X[X['Date_dt']>= '2018-01-01']
X_val = X[(X['Date_dt'] > '2017-01-01') & (X['Date_dt'] < '2018-01-01')]
y_train = y[X[X['Date_dt']<= '2017-01-01'].index]
y_test = y[X[X['Date_dt']>= '2018-01-01'].index]
y_val = y[X[(X['Date_dt'] > '2017-01-01') & (X['Date_dt'] < '2018-01-01')].index]
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape, X_val.shape, y_val.shape)

(940727, 11) (940727,) (106143, 11) (106143,) (121607, 11) (121607,)


In [118]:
encoding_len = {k: len(encodings[k]) + 1 for k in encodings}

In [120]:
def get_cat_num_split(df):
    X = {} 
    X['num'] = df.loc[:, num_cols].values
    X['num'] = np.reshape(X['num'], (X['num'].shape[0], 1, X['num'].shape[1]))
    for cat in categorical_cols:
        X[cat] = df.loc[:, cat].values
    return X


X_train_split = get_cat_num_split(X_train)
X_test_split = get_cat_num_split(X_test)
X_val_split = get_cat_num_split(X_val)



In [132]:
from keras.models import Model
from keras.layers import Input, Dense, Embedding, Concatenate, Flatten, BatchNormalization
from keras.models import Model
from keras.layers import Input, Dense, Embedding, Concatenate, Flatten, LSTM, Dropout, Reshape
from keras.losses import binary_crossentropy, mse
from keras.metrics import binary_accuracy
from keras.regularizers import l2
import keras.backend as K

DROPOUT_RATE = 0.2

cat_inputs = [Input(shape=[1], name=cat) for cat in categorical_cols]
embeddings = [Embedding(encoding_len[cat], embedding_lengths[i])(cat_inputs[i]) for i, cat in enumerate(categorical_cols)]
categorical_logits = Concatenate()([(cat_emb) for cat_emb in embeddings])
categorical_logits = LSTM(128, activation='relu', input_shape=(1, len(categorical_cols)), return_sequences=True,
                         kernel_regularizer=l2(1e-5), kernel_initializer='random_uniform')(categorical_logits)

numerical_inputs = Input(shape=(1, len(num_cols)), name='num')
numerical_logits = LSTM(256, activation='relu', input_shape=(1, len(num_cols)), return_sequences=True,
                        kernel_regularizer=l2(1e-5), kernel_initializer='random_uniform')(numerical_inputs)
numerical_logits = Dropout(DROPOUT_RATE)(numerical_logits)

logits = Concatenate()([numerical_logits,categorical_logits])
logits = LSTM(256, activation='relu', kernel_initializer='random_uniform')(logits)
out = Dense(1, activation='sigmoid', name='probability_layer')(logits)

model = Model(inputs = cat_inputs + [numerical_inputs], outputs=out)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics = ['binary_accuracy'])

In [133]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
ticker (InputLayer)             (None, 1)            0                                            
__________________________________________________________________________________________________
Sector (InputLayer)             (None, 1)            0                                            
__________________________________________________________________________________________________
year (InputLayer)               (None, 1)            0                                            
__________________________________________________________________________________________________
month (InputLayer)              (None, 1)            0                                            
__________________________________________________________________________________________________
num (Input

In [134]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

check_point = ModelCheckpoint('model.hdf5',verbose=True, save_best_only=True)
early_stop = EarlyStopping(patience=1,verbose=True)
model.fit(X_train_split, y_train,
          validation_data=(X_val_split, y_val),
          epochs=1,
          verbose=True,
          callbacks=[early_stop,check_point]) 

Train on 940727 samples, validate on 121607 samples
Epoch 1/1


  4928/940727 [..............................] - ETA: 38:13:18 - loss: 0.6937 - binary_accuracy: 0.562 - ETA: 19:23:19 - loss: 0.6937 - binary_accuracy: 0.531 - ETA: 13:05:40 - loss: 0.6936 - binary_accuracy: 0.531 - ETA: 9:56:36 - loss: 0.6937 - binary_accuracy: 0.515 - ETA: 8:04:20 - loss: 0.6937 - binary_accuracy: 0.51 - ETA: 6:49:39 - loss: 0.6938 - binary_accuracy: 0.50 - ETA: 5:56:18 - loss: 0.6937 - binary_accuracy: 0.51 - ETA: 5:16:14 - loss: 0.6936 - binary_accuracy: 0.51 - ETA: 4:45:01 - loss: 0.6935 - binary_accuracy: 0.53 - ETA: 4:19:39 - loss: 0.6933 - binary_accuracy: 0.54 - ETA: 3:59:34 - loss: 0.6932 - binary_accuracy: 0.54 - ETA: 3:42:47 - loss: 0.6931 - binary_accuracy: 0.54 - ETA: 3:28:37 - loss: 0.6930 - binary_accuracy: 0.55 - ETA: 3:16:25 - loss: 0.6931 - binary_accuracy: 0.54 - ETA: 3:05:38 - loss: 0.6932 - binary_accuracy: 0.54 - ETA: 2:56:03 - loss: 0.6932 - binary_accuracy: 0.54 - ETA: 2:47:19 - loss: 0.6930 - binary_accuracy: 0.54 - ETA: 2:39:34 - loss: 0.693

 10144/940727 [..............................] - ETA: 47:25 - loss: 0.6932 - binary_accuracy: 0.51 - ETA: 47:21 - loss: 0.6932 - binary_accuracy: 0.51 - ETA: 47:17 - loss: 0.6931 - binary_accuracy: 0.51 - ETA: 47:12 - loss: 0.6931 - binary_accuracy: 0.51 - ETA: 47:09 - loss: 0.6931 - binary_accuracy: 0.51 - ETA: 47:04 - loss: 0.6931 - binary_accuracy: 0.51 - ETA: 47:01 - loss: 0.6930 - binary_accuracy: 0.51 - ETA: 47:00 - loss: 0.6931 - binary_accuracy: 0.51 - ETA: 46:55 - loss: 0.6931 - binary_accuracy: 0.51 - ETA: 46:50 - loss: 0.6931 - binary_accuracy: 0.51 - ETA: 46:48 - loss: 0.6931 - binary_accuracy: 0.51 - ETA: 46:48 - loss: 0.6931 - binary_accuracy: 0.51 - ETA: 46:48 - loss: 0.6931 - binary_accuracy: 0.51 - ETA: 46:45 - loss: 0.6931 - binary_accuracy: 0.51 - ETA: 46:47 - loss: 0.6931 - binary_accuracy: 0.51 - ETA: 46:44 - loss: 0.6931 - binary_accuracy: 0.51 - ETA: 46:41 - loss: 0.6930 - binary_accuracy: 0.51 - ETA: 46:37 - loss: 0.6931 - binary_accuracy: 0.51 - ETA: 46:33 - lo

 15712/940727 [..............................] - ETA: 39:49 - loss: 0.6928 - binary_accuracy: 0.51 - ETA: 39:48 - loss: 0.6928 - binary_accuracy: 0.51 - ETA: 39:47 - loss: 0.6929 - binary_accuracy: 0.51 - ETA: 39:44 - loss: 0.6929 - binary_accuracy: 0.51 - ETA: 39:41 - loss: 0.6929 - binary_accuracy: 0.51 - ETA: 39:40 - loss: 0.6929 - binary_accuracy: 0.51 - ETA: 39:40 - loss: 0.6929 - binary_accuracy: 0.51 - ETA: 39:39 - loss: 0.6929 - binary_accuracy: 0.51 - ETA: 39:38 - loss: 0.6928 - binary_accuracy: 0.51 - ETA: 39:37 - loss: 0.6929 - binary_accuracy: 0.51 - ETA: 39:36 - loss: 0.6929 - binary_accuracy: 0.51 - ETA: 39:33 - loss: 0.6929 - binary_accuracy: 0.51 - ETA: 39:31 - loss: 0.6929 - binary_accuracy: 0.51 - ETA: 39:32 - loss: 0.6929 - binary_accuracy: 0.51 - ETA: 39:31 - loss: 0.6929 - binary_accuracy: 0.51 - ETA: 39:30 - loss: 0.6929 - binary_accuracy: 0.51 - ETA: 39:29 - loss: 0.6929 - binary_accuracy: 0.51 - ETA: 39:29 - loss: 0.6929 - binary_accuracy: 0.51 - ETA: 39:29 - lo

 21696/940727 [..............................] - ETA: 34:56 - loss: 0.6927 - binary_accuracy: 0.51 - ETA: 34:53 - loss: 0.6927 - binary_accuracy: 0.51 - ETA: 34:51 - loss: 0.6927 - binary_accuracy: 0.51 - ETA: 34:50 - loss: 0.6927 - binary_accuracy: 0.51 - ETA: 34:47 - loss: 0.6927 - binary_accuracy: 0.51 - ETA: 34:45 - loss: 0.6927 - binary_accuracy: 0.51 - ETA: 34:42 - loss: 0.6927 - binary_accuracy: 0.51 - ETA: 34:41 - loss: 0.6927 - binary_accuracy: 0.52 - ETA: 34:38 - loss: 0.6927 - binary_accuracy: 0.51 - ETA: 34:36 - loss: 0.6927 - binary_accuracy: 0.51 - ETA: 34:35 - loss: 0.6927 - binary_accuracy: 0.51 - ETA: 34:32 - loss: 0.6927 - binary_accuracy: 0.52 - ETA: 34:31 - loss: 0.6927 - binary_accuracy: 0.51 - ETA: 34:30 - loss: 0.6927 - binary_accuracy: 0.52 - ETA: 34:28 - loss: 0.6927 - binary_accuracy: 0.52 - ETA: 34:27 - loss: 0.6927 - binary_accuracy: 0.52 - ETA: 34:24 - loss: 0.6926 - binary_accuracy: 0.52 - ETA: 34:21 - loss: 0.6927 - binary_accuracy: 0.52 - ETA: 34:20 - lo

...

918880/940727 [============================>.] - ETA: 48s - loss: 0.6908 - binary_accuracy: 0.52 - ETA: 48s - loss: 0.6908 - binary_accuracy: 0.52 - ETA: 47s - loss: 0.6908 - binary_accuracy: 0.52 - ETA: 47s - loss: 0.6908 - binary_accuracy: 0.52 - ETA: 47s - loss: 0.6908 - binary_accuracy: 0.52 - ETA: 47s - loss: 0.6908 - binary_accuracy: 0.52 - ETA: 47s - loss: 0.6908 - binary_accuracy: 0.52 - ETA: 47s - loss: 0.6908 - binary_accuracy: 0.52 - ETA: 47s - loss: 0.6908 - binary_accuracy: 0.52 - ETA: 47s - loss: 0.6908 - binary_accuracy: 0.52 - ETA: 47s - loss: 0.6908 - binary_accuracy: 0.52 - ETA: 47s - loss: 0.6908 - binary_accuracy: 0.52 - ETA: 47s - loss: 0.6908 - binary_accuracy: 0.52 - ETA: 47s - loss: 0.6908 - binary_accuracy: 0.52 - ETA: 47s - loss: 0.6908 - binary_accuracy: 0.52 - ETA: 47s - loss: 0.6908 - binary_accuracy: 0.52 - ETA: 47s - loss: 0.6908 - binary_accuracy: 0.52 - ETA: 47s - loss: 0.6908 - binary_accuracy: 0.52 - ETA: 47s - loss: 0.6908 - binary_accuracy: 0.52 - E

924224/940727 [============================>.] - ETA: 38s - loss: 0.6908 - binary_accuracy: 0.52 - ETA: 38s - loss: 0.6908 - binary_accuracy: 0.52 - ETA: 38s - loss: 0.6908 - binary_accuracy: 0.52 - ETA: 38s - loss: 0.6908 - binary_accuracy: 0.52 - ETA: 38s - loss: 0.6908 - binary_accuracy: 0.52 - ETA: 38s - loss: 0.6908 - binary_accuracy: 0.52 - ETA: 38s - loss: 0.6908 - binary_accuracy: 0.52 - ETA: 38s - loss: 0.6908 - binary_accuracy: 0.52 - ETA: 38s - loss: 0.6908 - binary_accuracy: 0.52 - ETA: 38s - loss: 0.6908 - binary_accuracy: 0.52 - ETA: 38s - loss: 0.6908 - binary_accuracy: 0.52 - ETA: 38s - loss: 0.6908 - binary_accuracy: 0.52 - ETA: 38s - loss: 0.6908 - binary_accuracy: 0.52 - ETA: 38s - loss: 0.6908 - binary_accuracy: 0.52 - ETA: 38s - loss: 0.6908 - binary_accuracy: 0.52 - ETA: 37s - loss: 0.6908 - binary_accuracy: 0.52 - ETA: 37s - loss: 0.6908 - binary_accuracy: 0.52 - ETA: 37s - loss: 0.6908 - binary_accuracy: 0.52 - ETA: 37s - loss: 0.6908 - binary_accuracy: 0.52 - E

929600/940727 [============================>.] - ETA: 29s - loss: 0.6908 - binary_accuracy: 0.52 - ETA: 29s - loss: 0.6908 - binary_accuracy: 0.52 - ETA: 29s - loss: 0.6908 - binary_accuracy: 0.52 - ETA: 29s - loss: 0.6908 - binary_accuracy: 0.52 - ETA: 29s - loss: 0.6908 - binary_accuracy: 0.52 - ETA: 29s - loss: 0.6908 - binary_accuracy: 0.52 - ETA: 28s - loss: 0.6908 - binary_accuracy: 0.52 - ETA: 28s - loss: 0.6908 - binary_accuracy: 0.52 - ETA: 28s - loss: 0.6908 - binary_accuracy: 0.52 - ETA: 28s - loss: 0.6908 - binary_accuracy: 0.52 - ETA: 28s - loss: 0.6908 - binary_accuracy: 0.52 - ETA: 28s - loss: 0.6908 - binary_accuracy: 0.52 - ETA: 28s - loss: 0.6908 - binary_accuracy: 0.52 - ETA: 28s - loss: 0.6908 - binary_accuracy: 0.52 - ETA: 28s - loss: 0.6908 - binary_accuracy: 0.52 - ETA: 28s - loss: 0.6908 - binary_accuracy: 0.52 - ETA: 28s - loss: 0.6908 - binary_accuracy: 0.52 - ETA: 28s - loss: 0.6908 - binary_accuracy: 0.52 - ETA: 28s - loss: 0.6908 - binary_accuracy: 0.52 - E

935552/940727 [============================>.] - ETA: 19s - loss: 0.6908 - binary_accuracy: 0.52 - ETA: 19s - loss: 0.6908 - binary_accuracy: 0.52 - ETA: 19s - loss: 0.6908 - binary_accuracy: 0.52 - ETA: 19s - loss: 0.6908 - binary_accuracy: 0.52 - ETA: 19s - loss: 0.6908 - binary_accuracy: 0.52 - ETA: 19s - loss: 0.6908 - binary_accuracy: 0.52 - ETA: 19s - loss: 0.6908 - binary_accuracy: 0.52 - ETA: 19s - loss: 0.6908 - binary_accuracy: 0.52 - ETA: 19s - loss: 0.6908 - binary_accuracy: 0.52 - ETA: 19s - loss: 0.6908 - binary_accuracy: 0.52 - ETA: 19s - loss: 0.6908 - binary_accuracy: 0.52 - ETA: 19s - loss: 0.6908 - binary_accuracy: 0.52 - ETA: 19s - loss: 0.6908 - binary_accuracy: 0.52 - ETA: 18s - loss: 0.6908 - binary_accuracy: 0.52 - ETA: 18s - loss: 0.6908 - binary_accuracy: 0.52 - ETA: 18s - loss: 0.6908 - binary_accuracy: 0.52 - ETA: 18s - loss: 0.6908 - binary_accuracy: 0.52 - ETA: 18s - loss: 0.6908 - binary_accuracy: 0.52 - ETA: 18s - loss: 0.6908 - binary_accuracy: 0.52 - E

940727/940727 [==============================] - ETA: 9s - loss: 0.6908 - binary_accuracy: 0.528 - ETA: 9s - loss: 0.6908 - binary_accuracy: 0.528 - ETA: 9s - loss: 0.6908 - binary_accuracy: 0.528 - ETA: 8s - loss: 0.6908 - binary_accuracy: 0.528 - ETA: 8s - loss: 0.6908 - binary_accuracy: 0.528 - ETA: 8s - loss: 0.6908 - binary_accuracy: 0.528 - ETA: 8s - loss: 0.6908 - binary_accuracy: 0.528 - ETA: 8s - loss: 0.6908 - binary_accuracy: 0.528 - ETA: 8s - loss: 0.6908 - binary_accuracy: 0.528 - ETA: 8s - loss: 0.6908 - binary_accuracy: 0.528 - ETA: 8s - loss: 0.6908 - binary_accuracy: 0.528 - ETA: 8s - loss: 0.6908 - binary_accuracy: 0.528 - ETA: 8s - loss: 0.6908 - binary_accuracy: 0.528 - ETA: 8s - loss: 0.6908 - binary_accuracy: 0.528 - ETA: 8s - loss: 0.6907 - binary_accuracy: 0.528 - ETA: 8s - loss: 0.6907 - binary_accuracy: 0.528 - ETA: 8s - loss: 0.6907 - binary_accuracy: 0.528 - ETA: 8s - loss: 0.6907 - binary_accuracy: 0.528 - ETA: 8s - loss: 0.6907 - binary_accuracy: 0.528 - E

In [124]:
#pred = model.predict_proba(X_test_split)

In [135]:
scores = model.evaluate(X_test_split, y_test, verbose=0)

[0.6954350265177347, 0.49155384716423095]

In [136]:
print("Accuracy: %.2f%%" % (scores*100))

TypeError: must be real number, not list

In [ ]:
scores1 = model.evaluate(X_val_split, y_val, verbose = 0)
print("Accuracy: %.2f%%" % (scores1*100))

In [ ]:
y_classes = pred.argmax(axis=-1)

In [ ]:
scores1 = model.evaluate(X_train_split, y_train, verbose = 0)
print("Accuracy: %.2f%%" % (scores1*100))